In [4]:
import boto3
from boto3.session import Session

session = Session(profile_name='cristal-management')
# Create an AWS Config client
config_client = session.client('config')
# Retrieve the status of all configuration recorders


def is_aws_config_enabled(client):
    response = client.describe_configuration_recorder_status()
    recorders_status = response['ConfigurationRecordersStatus']

    if not recorders_status:
        return False
    else:
        # Check each recorder's status
        for recorder_status in recorders_status:
            if recorder_status['recording'] and recorder_status.get('lastStatus', '') == 'SUCCESS':
                return True
            else:
                return False
# response = config_client.describe_configuration_recorder_status()
# recorders_status = response['ConfigurationRecordersStatus']

# if not recorders_status:
#     print("AWS Config is not enabled: No configuration recorders found.")
# else:
#     # Check each recorder's status
#     for recorder_status in recorders_status:
#         if recorder_status['recording'] and recorder_status.get('lastStatus', '') == 'SUCCESS':
#             print(f"AWS Config is enabled and recording. Recorder name: {recorder_status['name']}")
#         else:
#             print(f"AWS Config recorder {recorder_status['name']} is not recording or has issues.")

In [7]:
# from .tools import Tools
# from .constants import CT_EXECUTION_ROLE_NAME

from right_start_tools.tools import Tools
from right_start_tools.constants import CT_EXECUTION_ROLE_NAME

t = Tools(session)
regions = ["us-east-1", "us-west-2"]

root_id = t.org.get_root_id()
structure = t.org.get_org_structure(root_id)
accounts = structure.all_accounts()
for account in accounts:
    try:
        credentials = t.sts.assume_role_and_get_credentials(
            account.id, CT_EXECUTION_ROLE_NAME
        )
        acc_session = Session(
            aws_access_key_id=credentials["aws_access_key_id"],
            aws_secret_access_key=credentials["aws_secret_access_key"],
            aws_session_token=credentials["aws_session_token"],
        )
        for region in regions:
            aws_config_enabled = is_aws_config_enabled(acc_session.client('config', region_name=region))
            print(f"account {account.id} | region: {region} | config: {'+' if aws_config_enabled else '-'}")
    except Exception as e:
        print(f"account {account.id} | error: {e}")
        continue

account 637423411767 | region: us-east-1 | config: +
account 637423411767 | region: us-west-2 | config: +
account 058264379639 | region: us-east-1 | config: +
account 058264379639 | region: us-west-2 | config: +
account 975050101446 | region: us-east-1 | config: +
account 975050101446 | region: us-west-2 | config: +
account 058264343716 | region: us-east-1 | config: +
account 058264343716 | region: us-west-2 | config: +
account 851725405272 | region: us-east-1 | config: +
account 851725405272 | region: us-west-2 | config: +
account 058264102876 | region: us-east-1 | config: +
account 058264102876 | region: us-west-2 | config: +
account 533267202684 | region: us-east-1 | config: +
account 533267202684 | region: us-west-2 | config: +
account 975050137657 | error: An error occurred (AccessDenied) when calling the AssumeRole operation: User: arn:aws:sts::339713178213:assumed-role/AWSReservedSSO_ManagementAccountAdminAccess_cdffadf10d0395e8/alexey.eremin@clearwayhealth.com is not authorized